In [2]:
# %load_ext autoreload
# %autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

from utils import bedrock, print_ww

1. BedRock Client 생성

In [4]:
import json
import boto3
from termcolor import colored
from pprint import  pprint
from utils.bedrock import bedrock_info # class method

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

boto3_bedrock_cli = bedrock.get_bedrock_client(
  assumed_role = os.environ.get("BEDROCK_ASSUME_ROLE",None),
  endpoint_url = os.environ.get("BEDROCK_ENDPOINT_URL", None),
  region = os.environ.get("AWS_DEFAULT_REGION",None)
)
print(colored("\n== FM list =="))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM list ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


2. LangChain Integration을 사용하여 Bedrock Client 호출

In [5]:
from langchain.llms import Bedrock

inference_modifier = {
  'max_tokens_to_sample': 4096,
  'temperature': 0.5,
  'top_k': 250,
  'top_p': 1,
  'stop_sequences': ["\n\nHuman"]
}

textgen_llm = Bedrock(
  model_id = 'anthropic.claude-v2',
  client = boto3_bedrock_cli,
  model_kwargs = inference_modifier
)

3. LangChain 사용자 정의 프롬프트 템플릿 생성

In [11]:
from langchain import PromptTemplate

multi_var_prompt = PromptTemplate(
  input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"],
  template="""서비스 관리자 {customerServiceManager}가 {customerName}에게 보내는 사과 이메일을 작성합니다.
  고객으로부터 받은 다음 피드백에 대한 응답: {feedbackFromCustomer}"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="권율",
                                  customerName="이순신",
                                  feedbackFromCustomer="""안녕하세요 권율님,
     귀하의 고객 지원팀에 전화했을 때의 최근 경험에 매우 실망했습니다.
     나는 즉시 전화를 받을 것으로 예상했지만 전화를 받는 데 3일이 걸렸습니다.
     문제를 해결하기 위한 첫 번째 제안이 올바르지 않았습니다. 결국 문제는 3일 만에 해결 되었습니다.
     우리는 제공된 응답에 매우 만족하지 않으며 다른 곳에서 비즈니스를 수행하는 것을 고려할 수 있습니다. 
     """
     )

In [12]:
print("prompt: \n", prompt)

prompt: 
 서비스 관리자 권율가 이순신에게 보내는 사과 이메일을 작성합니다.
  고객으로부터 받은 다음 피드백에 대한 응답: 안녕하세요 권율님,
     귀하의 고객 지원팀에 전화했을 때의 최근 경험에 매우 실망했습니다.
     나는 즉시 전화를 받을 것으로 예상했지만 전화를 받는 데 3일이 걸렸습니다.
     문제를 해결하기 위한 첫 번째 제안이 올바르지 않았습니다. 결국 문제는 3일 만에 해결 되었습니다.
     우리는 제공된 응답에 매우 만족하지 않으며 다른 곳에서 비즈니스를 수행하는 것을 고려할 수 있습니다. 
     


In [10]:
HUMAN = "\n\nHuman:"
ASSISTANT = "\n\nAssistant:"
prompt = HUMAN + prompt + ASSISTANT
# print(prompt)



Human:서비스 관리자 권율가 이순신에게 보내는 사과 이메일을 작성합니다.
  고객으로부터 받은 다음 피드백에 대한 응답: 안녕하세요 권율님,
     귀하의 고객 지원팀에 전화했을 때의 최근 경험에 매우 실망했습니다.
     나는 즉시 전화를 받을 것으로 예상했지만 전화를 받는 데 3일이 걸렸습니다.
     문제를 해결하기 위한 첫 번째 제안이 올바르지 않았습니다. 결국 문제는 3일 만에 해결 되었습니다.
     우리는 제공된 응답에 매우 만족하지 않으며 다른 곳에서 비즈니스를 수행하는 것을 고려할 수 있습니다. 
     
  

Assistant:


In [13]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

Our prompt has 274 tokens


4. PROMPT 실행

In [15]:
response = textgen_llm(prompt)
email = response[response.index('\n')+1:]
print_ww(colored(email, 'green'))


저는 최근 고객님과의 상호작용에서 발생한 불편을 진심으로 사과드립니다. 고객님께 신속하고 정확한 서비스를 제공하는 것이 우리의 최우선 과제입니다.

3일이 넘게 기다려야 했다는 점과 처음 제안한 해결책이 적절하지 않았다는 점에서 고객님께 실망을 드린 점 진심으로 죄송합니다. 이는 우리의 서비스 수준에 대한 고객님의 기대에
미치지 못한 것이며, 저희 서비스 품질 개선을 위해 귀중한 피드백으로 여기고 있습니다.

앞으로 이러한 일이 다시 발생하지 않도록 문제 해결 프로세스를 개선하고, 고객님의 요구에 더욱 신속하고 정확하게 대응할 수 있는 교육을 직원들에게 시행하겠습니다.

고객님의 지속적인 성원에 감사드리며, 더 나은 서비스로 보답할 수 있도록 노력하겠습니다. 언제든지 피드백을 주십시오.

감사합니다.
